<a href="https://colab.research.google.com/github/Deegayu-Mihiraj/Bank-Marketing-ML-Project/blob/main/ML%20CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

Mount Google Drive and import dataset

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
bankdataset = pd.read_csv('/content/drive/MyDrive/Machine Learning/CW/bank-full.csv', sep=';')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


***Data Preprocessing***

In [ ]:
# Drop 'duration' column for realistic predictive modeling
bankdataset.drop(columns=['duration'], inplace=True)

In [ ]:
bankdataset.head()

In [45]:
bankdataset.shape

(45211, 17)

In [46]:
bankdataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [47]:
bankdataset.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [61]:
# Check for missing values in the entire dataset
missing_values = bankdataset.isnull().sum()
print(missing_values)

age          0
balance      0
day          0
duration     0
campaign     0
pdays        0
previous     0
job          0
marital      0
education    0
default      0
housing      0
loan         0
contact      0
month        0
poutcome     0
y            0
dtype: int64


In [51]:
bankdataset.isin(['unknown']).sum()


,0
age,0
job,288
marital,0
education,1857
default,0
balance,0
housing,0
loan,0
contact,13020
day,0


In [52]:
# List of columns where "unknown" should be replaced with the mode
selected_columns = ['job', 'education']

# Replace "unknown" with the mode for the selected columns
for col in selected_columns:
    mode = bankdataset[col].mode()[0]
    bankdataset[col] = bankdataset[col].replace('unknown', mode)


In [53]:
integer_columns = bankdataset.select_dtypes(include=['int64'])
integer_columns

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0
...,...,...,...,...,...,...,...
45206,51,825,17,977,3,-1,0
45207,71,1729,17,456,2,-1,0
45208,72,5715,17,1127,5,184,3
45209,57,668,17,508,4,-1,0


In [54]:
string_columns = bankdataset.select_dtypes(include=['object'])
string_columns

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,may,unknown,no
1,technician,single,secondary,no,yes,no,unknown,may,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown,no
3,blue-collar,married,secondary,no,yes,no,unknown,may,unknown,no
4,blue-collar,single,secondary,no,no,no,unknown,may,unknown,no
...,...,...,...,...,...,...,...,...,...,...
45206,technician,married,tertiary,no,no,no,cellular,nov,unknown,yes
45207,retired,divorced,primary,no,no,no,cellular,nov,unknown,yes
45208,retired,married,secondary,no,no,no,cellular,nov,success,yes
45209,blue-collar,married,secondary,no,no,no,telephone,nov,unknown,no


In [55]:
# Remove punctuation
import string

string_columns = bankdataset.select_dtypes(include=['object'])
bankdataset[string_columns.columns] = string_columns.apply(
    lambda col: col.str.replace(f"[{string.punctuation}]", "", regex=True)
)


In [60]:
# Encoding categorical variables
label_encoders = {}
for col in string_columns.columns:
    le = LabelEncoder()
    bankdataset[col] = le.fit_transform(bankdataset[col])
    label_encoders[col] = le

# Encoding target variable 'y'
bankdataset['y'] = label_encoders['y'].transform(bankdataset['y'])

Columns containing 'unknown': []


In [ ]:
# Scaling numeric features
scaler = StandardScaler()
integer_columns_scaled = scaler.fit_transform(bankdataset[integer_columns.columns])
integer_columns_scaled_df = pd.DataFrame(integer_columns_scaled, columns=integer_columns.columns)

# Combine scaled numeric features with encoded categorical features
bankdataset_preprocessed = pd.concat([integer_columns_scaled_df, bankdataset[string_columns.columns]], axis=1)


In [ ]:
# Splitting the dataset into train and test sets
X = bankdataset_preprocessed.drop(columns=['y'])
y = bankdataset_preprocessed['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
